In [1]:
import pandas as pd
import json
from pathlib import Path
import numpy as np


file_path = "C:/Users/91901/Downloads/synthetic_fb_ads_undergarments.csv" 
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please check your path.")
    raise

# Robust Column Cleaning and Date Conversion
df.columns = df.columns.str.lower().str.strip()
df['date'] = pd.to_datetime(df['date'])

print("Setup Complete: Full DataFrame (df) is loaded for creative analysis.")

Setup Complete: Full DataFrame (df) is loaded for creative analysis.


In [3]:
LOW_CTR_THRESHOLD = 1.5 
MIN_IMPRESSIONS = 1000 

creative_agg_df = df.groupby(['adset_name', 'creative_message']).agg(
    total_impressions=('impressions', 'sum'),
    total_clicks=('clicks', 'sum'),
    latest_creative_type=('creative_type', 'last') 
).reset_index()


creative_agg_df['creative_ctr'] = (
    creative_agg_df['total_clicks'] / creative_agg_df['total_impressions']
) * 100


low_ctr_df = creative_agg_df[
    (creative_agg_df['creative_ctr'] < LOW_CTR_THRESHOLD) & 
    (creative_agg_df['total_impressions'] > MIN_IMPRESSIONS) 
].sort_values(by='creative_ctr', ascending=True)


top_low_ctr_creatives = low_ctr_df.head(5)

print(f"--- Top {len(top_low_ctr_creatives)} Creatives Identified (Input for Generator) ---")
print(top_low_ctr_creatives[['adset_name', 'creative_message', 'creative_ctr']].set_index('adset_name'))


low_ctr_summary = top_low_ctr_creatives[['creative_message', 'latest_creative_type', 'creative_ctr']].to_dict('records')

--- Top 5 Creatives Identified (Input for Generator) ---
                                               creative_message  creative_ctr
adset_name                                                                   
Adset-5 ATC   Doctors recommend breathable cotton — meet our...           0.0
Adset-3 ATC   Seamless confidence for every day — new men bo...           0.0
Adset-1 LAL1  Doctors recommend breathable microfiber — meet...           0.0
Adset-4 LAL1  Push comfort, not wires — everyday men inner v...           0.0
Adset-4 WC    Wire‑free ease, cloud‑soft cups — women bras t...           0.0


In [5]:
def generate_creatives(low_ctr_data: list) -> list:
    """
    Simulates the Creative Improvement Generator Agent. 
    The output is grounded in the existing creative messaging data.
    """
    
    creative_recs = []
    
    
    for i, item in enumerate(low_ctr_data):
        recs = {
            "creative_id": f"R{i+1}",
            "original_message_context": item['creative_message'],
            "original_ctr": f"{item['creative_ctr']:.2f}%",
            "new_message_concept": "Shift to high-urgency, benefit-focused messaging.",
            "suggested_headlines": [
                "Time's Running Out: Limited Stock Available!",
                "Join Thousands: See Why Everyone is Loving This Product"
            ],
            "suggested_message_body": "Emphasize problem-solving and offer a limited-time incentive to boost clicks.",
            "suggested_cta": "Claim Offer",
            "reasoning": f"Original {item['latest_creative_type']} was too descriptive; low CTR suggests the message lacked a strong hook."
        }
        creative_recs.append(recs)
        
    return creative_recs

creative_recommendations = generate_creatives(low_ctr_summary)


import os
from pathlib import Path

output_dir = 'reports' 
output_path_creatives = Path(output_dir) / 'creatives.json' 

Path(output_dir).mkdir(exist_ok=True) 

with open(output_path_creatives, 'w') as f:
    json.dump(creative_recommendations, f, indent=4)

print(f"\n✅ Creative Recommendations Saved to {output_path_creatives}")


✅ Creative Recommendations Saved to reports\creatives.json
